# 검색 알고리즘 실험
> 특정 문서를 입력받아 코사인 유사도를 기반으로 유사한 문서를 반환하는 실험을 진행해보자.

1. TF-IDF + NMF
1. Word2vec
1. Fasttext

## Import

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.preprocessing import Normalizer
from konlpy.tag import Mecab
from gensim.models import Word2Vec
from gensim.models import FastText

## Data
>sourced by momtalk(https://www.momtalk.kr/talk/list/all)

1. `df1`: 육아정보 포스팅
1. `df2`: 유아용품 포스팅
1. `df` : 위의 두 데이터를 합친 것

In [2]:
df = pd.read_csv('data/text_for_word2vec.csv')
df = df.drop(index=df[df['text'] == ""].index)
df = df.dropna()
df = df.reset_index(drop=True)

In [3]:
def tokenize(text):
    mecab = Mecab()
    tokens = mecab.pos(text)
    key_tokens = [token[0] for token in tokens 
                 if ('NNG' in token or 'NNP' in token or 'VA' in token or 'VV' in token)]
    key_tokens = ' '.join(key_tokens)
    return key_tokens

In [4]:
df['tokens'] = df['text'].apply(tokenize)

In [5]:
df.tail()

,text,tokens
19500,토끼띠의 단점,토끼띠 단점
19501,국가영어능력평가 준비 올해부터 본격적으로 시행되는 국가영어능력평가에 대해서 소개해드...,국가 영어 능력 평가 준비 올해 본격 시행 국가 영어 능력 평가 소개 국가 영어 능...
19502,워킹맘이든 전업맘이든 아이들에게 관심과 사랑을 쏟아야 하는 것은 분명합니다 평소 아...,워킹 맘 전업 맘 아이 관심 사랑 쏟 하 평소 아이 생각 가지 관심 있 하 일 흥미...
19503,검사결과 지민이의 강점지능은 언어지능과 음악지능였습니다 엄마는 지민이가 집에서나 학...,검사 결과 지민 강점 지능 언어 지능 음악 지능 엄마 지민 집 학교 말 좋아하 질문...
19504,진로상담센터 교육 컨설팅 아이들의 진로는 중·고등학생 때 뿐 아니라 초등학생때부터 ...,진로 상담 센터 교육 컨설팅 아이 진로 중 고등학생 때 초등학생 때 고민 필요 있 ...


## TF-IDF

In [6]:
%%time
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(df['tokens'])

print(vectors.shape)

(19505, 55450)
CPU times: user 2.01 s, sys: 11 ms, total: 2.02 s
Wall time: 1.78 s


## NMF(Non-negative Matrix Factorization)
- sparse coding처럼 dictionary와 encoded vector 학습
- parameter가 0보다 큰 특성으로 feature matrix가 원본 데이터들과 어떠한 관계를 가지는지 해석이 가능
- 연산에 약간의 시간 소요

In [7]:
%%time
vector_array = vectors.toarray()
nmf = NMF(n_components=40)
nmf.fit(vector_array)
features = nmf.transform(vector_array)

CPU times: user 10min 5s, sys: 7.37 s, total: 10min 12s
Wall time: 1min 22s


## Normalize extracted feature set
- 0과 1사이의 값으로 정규화하여 feature vector의 길이를 1로 고정

In [8]:
normalizer = Normalizer()
norm_features=normalizer.fit_transform(features)

print(norm_features[0:2])

[[0.80141411 0.         0.         0.         0.         0.
  0.         0.         0.         0.14065164 0.         0.
  0.         0.         0.00599881 0.         0.42455168 0.
  0.20764205 0.         0.         0.30783041 0.         0.13631752
  0.         0.         0.         0.00774208 0.         0.
  0.         0.         0.00784926 0.         0.         0.
  0.         0.03306998 0.         0.        ]
 [0.45945546 0.         0.         0.0744715  0.00122911 0.
  0.         0.         0.         0.         0.         0.04410036
  0.         0.         0.         0.         0.         0.02947643
  0.08410182 0.         0.10855309 0.56382027 0.         0.66579624
  0.         0.         0.         0.         0.01658894 0.
  0.         0.         0.01514771 0.         0.         0.
  0.         0.         0.         0.        ]]


In [9]:
df_features = pd.DataFrame(norm_features,index=df.index.tolist())
df_features.tail()

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
19500,0.039598,0.000000,0.000000,0.032457,0.0,0.0,0.127449,0.0,0.000000,0.000000,...,0.15643,0.124731,0.000000,0.740843,0.000000,0.0,0.000000,0.000000,0.330615,0.0
19501,0.008377,0.000000,0.004856,0.013404,0.0,0.0,0.056530,0.0,0.000000,0.896918,...,0.00000,0.000000,0.000000,0.114638,0.106493,0.0,0.303353,0.041320,0.042709,0.0
19502,0.559308,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.628985,0.219744,...,0.00000,0.000000,0.000000,0.000000,0.000947,0.0,0.020305,0.038971,0.000000,0.0
19503,0.063820,0.000000,0.059543,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.506146,...,0.00000,0.000000,0.193245,0.000000,0.000000,0.0,0.550234,0.000000,0.000000,0.0
19504,0.612080,0.165697,0.036810,0.000000,0.0,0.0,0.000000,0.0,0.000007,0.499052,...,0.00000,0.000000,0.005340,0.000000,0.000000,0.0,0.328638,0.000000,0.000000,0.0


`df`의 맨 첫번째 텍스트를 정규화 한 40차원 벡터의 값

In [10]:
print(df.loc[0].text)
article = df_features.loc[0]
len(article)

6살 남아 입니다 회사 복직 후 3살부터 주중에는 외할머니댁에서 2살 4살 터울의 사촌 형 누나와 함께 자랐어요 금요일 저녁에 데려와 주말에는 엄마 아빠와 생활하고 있구요 7세가 되는 내년이라도 아이를 데려오자는 아이 아빠의 의견과 유치원과 태권도 등 패턴이 적응되었고 형 누나들과 더 자랄 수 있게 초등학교 입학시기에 데리고 가는게 낫지 않겠냐는 외할머니 의견이 다릅니다 6세가 된 아이의 교육과 생활습관 형성이 이제는 할머니댁에서는 어려울 것 같다는 생각이 들고 하루하루 생각이 크고 자라는 아이를 주말에만 보는 저 또한 놀랄 때가 많거든요 하루라도 빨리 데려와야 할까요 초등학교 입학시기에 데려오는 편이 아이에게 좋을까요 매우 어려운 질문입니다 먼저 아이가 현재 잘 적응하고 있고 주변 양육환경 역시 아이가 성장하기에 별다른 문제가 없다 하니 고민이 더욱 크시리라 생각됩니다 우선 되어야하는 것은 아이의 의견입니다 6살이면 이러한 문제에 대하여 스스로 생각하고 의견을 전달할 수 있으므로 먼저 아이의 의견을 들어 주시기 바랍니다 아이는 부모님과 함께 사는 것이 좋습니다 그러나 부모님께서 두분 모두 직장에 나가시니 유치원 하원 후 아이를 돌보아 주실 분이 현재 함께 생활하고 계시는 할머님이시면 재고의 여지 없이 데려 오시는 것을 권유 드립니다 그러나 하원 후 할머님의 돌봄이 여의치 못할 경우 돌보아 줄 대상이 당분간 할머님댁에서 시간제로나마 아이를 돌보아 주어 아이와의 상호작용을 충분히 가진 후에 아이가 집으로 왔을 때 유치원 하원 후에 계속 돌보아 줄 수 있으면 좋습니다 이와 같이 먼저 아이가 집으로 온 후 부모님께서 퇴근하시기 전 까지 돌보아 줄 사람을 알아보시고 데려 오시는 것이 좋다는 생각이 듭니다


40

In [11]:
similarities=df_features.dot(article)
top=similarities.nlargest(10)

texts = df.loc[top.index]['text'].tolist()
i = 0
for text in texts:
    df_idx = top.index[i]
    print('TITLE :'+str(df_idx)+" Similarities:"+ str(top[df_idx]))
    print(text+'\n')
    i = i + 1

TITLE :0 Similarities:0.9999999999999999
6살 남아 입니다 회사 복직 후 3살부터 주중에는 외할머니댁에서 2살 4살 터울의 사촌 형 누나와 함께 자랐어요 금요일 저녁에 데려와 주말에는 엄마 아빠와 생활하고 있구요 7세가 되는 내년이라도 아이를 데려오자는 아이 아빠의 의견과 유치원과 태권도 등 패턴이 적응되었고 형 누나들과 더 자랄 수 있게 초등학교 입학시기에 데리고 가는게 낫지 않겠냐는 외할머니 의견이 다릅니다 6세가 된 아이의 교육과 생활습관 형성이 이제는 할머니댁에서는 어려울 것 같다는 생각이 들고 하루하루 생각이 크고 자라는 아이를 주말에만 보는 저 또한 놀랄 때가 많거든요 하루라도 빨리 데려와야 할까요 초등학교 입학시기에 데려오는 편이 아이에게 좋을까요 매우 어려운 질문입니다 먼저 아이가 현재 잘 적응하고 있고 주변 양육환경 역시 아이가 성장하기에 별다른 문제가 없다 하니 고민이 더욱 크시리라 생각됩니다 우선 되어야하는 것은 아이의 의견입니다 6살이면 이러한 문제에 대하여 스스로 생각하고 의견을 전달할 수 있으므로 먼저 아이의 의견을 들어 주시기 바랍니다 아이는 부모님과 함께 사는 것이 좋습니다 그러나 부모님께서 두분 모두 직장에 나가시니 유치원 하원 후 아이를 돌보아 주실 분이 현재 함께 생활하고 계시는 할머님이시면 재고의 여지 없이 데려 오시는 것을 권유 드립니다 그러나 하원 후 할머님의 돌봄이 여의치 못할 경우 돌보아 줄 대상이 당분간 할머님댁에서 시간제로나마 아이를 돌보아 주어 아이와의 상호작용을 충분히 가진 후에 아이가 집으로 왔을 때 유치원 하원 후에 계속 돌보아 줄 수 있으면 좋습니다 이와 같이 먼저 아이가 집으로 온 후 부모님께서 퇴근하시기 전 까지 돌보아 줄 사람을 알아보시고 데려 오시는 것이 좋다는 생각이 듭니다

TITLE :1761 Similarities:0.9512030126923522
아이가 갑자기 학원을 안 가겠대요 이미지 아이아트그만둬도 될까 아이가 갑자기 학원을 안 가겠대요 아이가 너무 원해

### 가상의 키워드를 입력하여 유사도 계산해보기
- 이번에는 챗봇처럼 짧은 문장(아마 키워드 위주)을 입력받아 유사도 계산을 해보자.
- Vocabulary에 없는 단어를 입력받을 경우 에러가 발생하는 문제가 있다.

In [12]:
np.array([[1, 2]]).shape

(1, 2)

In [13]:
%%time
tokenizer = Mecab()

input_text = "중고 유모차 어떤 브랜드가 좋을까요? 추천 부탁해요 ㅇㅅㅇ!"

tokens = tokenizer.morphs(input_text)
input_vector = []

for token in tokens:
    try:
        word_vector = vectorizer.transform([token])
        word_vector = word_vector.toarray().reshape(-1,)
        input_vector.append(word_vector)
    except KeyError:
        print("{}는 아직 모르겠습니다.".format(token))
        continue

input_vector = np.array(input_vector)
input_vector = np.sum(input_vector, axis=0)
input_vector = input_vector.reshape(1, -1)

input_vector = nmf.transform(input_vector)
input_vector = normalizer.transform(input_vector)
input_vector = input_vector.reshape(-1,)

similarities=df_features.dot(input_vector)
top=similarities.nlargest(5)

texts = df.loc[top.index]['text'].tolist()

i = 0
for text in texts:
    df_idx = top.index[i]
    print('\n'+'TITLE :'+str(df_idx)+" Similarities:"+ str(top[df_idx]))
    print(text+'\n')
    i = i+1


TITLE :4247 Similarities:0.9985353074270824
56년 전통의 이태리 프리미엄 유아용품 브랜드 잉글레시나가 신제품 출시를 맞아 9월 한 달 간 프로모션을 진행한다고 6일 밝혔다 이번 프로모션은 9월 동안 잉글레시나 유모차 또는 카시트를 구매하는 고객을 대상으로 진행하며 추첨을 통해 LG 퓨리케어 미니 공기청정기 밴밧 플레이아치 유모차 모빌 시크 카시트 안전벨트 인형 스타벅스 아메리카노 던킨도너츠 먼치킨 등 다양한 선물을 증정할 예정이다 온라인과 오프라인을 불문하고 잉글레시나 공식 거래처를 통해 유모차와 카시트를 구매했다면 자동으로 응모된다 당첨자 발표는 10월 3일로 예정돼 있다 잉글레시나 관계자는 지난해 론칭한 토들러 카시트 뉴튼에 이어 최신 안전 기준으로 만들어진 다윈 아이사이즈 카시트 그리고 확장형 주니어카시트 토로메오까지 총 세가지 카시트를 론칭하며 라인업을 갖추게 됐다며 이를 기념해 9월 한 달 간 잉글레시나 구매 고객을 위한 프로모션을 진행하니 소비자들의 많은 참여 바란다고 전했다 한편 잉글레시나는 한국소비자만족지수 1위를 4년 연속 수상할 만큼 국내 소비자들의 깊은 신뢰와 사랑을 받고 있다 특히 이번 해에는 스테디셀러 절충형 유모차 트릴로지를 비롯해 디럭스 유모차 앱티카 휴대용 유모차 퀴드에 이어 유럽 최신의 안전 인증을 적용한 다윈 아이사이즈 카시트까지 출시해 소비자들의 주목을 받았다


TITLE :10945 Similarities:0.9982091419851686
페도라 유모차 카시트는 세계 3대 유아 박람회 중 하나인 독일 쾰른 전시회KindJugend 2018 소개에 앞서 오는 23일부터 26일까지 열리는 코엑스 베페 베이비 페어에 참가해 카본 유모차 익스트림 X1과 카본 카시트 에볼루션 V1을 사전 공개한다 세계 최초로 카본 소재를 접목한 유모차 익스트림 X1 카본 카시트 에볼루션 V1은 카본 유모차 카시트는 지난 7월 세계 3대 유아용품 박람 중 하나인 CBMEChildren Baby Maternity 

```
tokenizer = Mecab()

while True:
    input_text = input("Input text: ")
    if input_text == "exit":
        print("Bye")
        break
        
    tokens = tokenizer.morphs(input_text)
    input_vector = []
    
    for token in tokens:
        try:
            word_vector = vectorizer.transform([token])
            word_vector = word_vector.toarray().reshape(-1,)
            input_vector.append(word_vector)
        except KeyError:
            print("{}는 아직 모르겠습니다.".format(token))
            continue

    input_vector = np.array(input_vector)
    input_vector = np.sum(input_vector, axis=0)
    input_vector = input_vector.reshape(1, -1)
    
    input_vector = nmf.transform(input_vector)
    input_vector = normalizer.transform(input_vector)
    input_vector = input_vector.reshape(-1,)
    
    similarities=df_features.dot(input_vector)
    top=similarities.nlargest(5)

    texts = df.loc[top.index]['text'].tolist()

    i = 0
    for text in texts:
        df_idx = top.index[i]
        print('\n'+'TITLE :'+str(df_idx)+" Similarities:"+ str(top[df_idx]))
        print(text+'\n')
        i = i+1
```

---

## Word2Vec
- 위의 TF-IDF 임베딩이 아닌 Word2Vec 임베딩으로 실험해보자.
- 음수의 값이 포함되서는 안되는 NMF는 활용할 수 없다. 어차피 sparse하지 않으면서 TF-IDF에 비해 훨씬 저차원인 성격을 지닌 word2vec 임베딩은 바로 유사도를 계산할 수 있어 더욱 편리하게 활용할 수 있을 것이라 판단된다.

In [14]:
w2v_model = Word2Vec.load('data/tokmom_word2vec_all')
df = pd.read_csv('data/text_for_word2vec.csv')
df = df.drop(index=df[df['text'] == ""].index)
df = df.dropna()
df = df.reset_index(drop=True)

In [15]:
def tokenize(text):
    tokenizer = Mecab()
    text = tokenizer.morphs(text)
    return text

In [16]:
df['tokenized_text'] = df['text'].apply(tokenize)

In [17]:
def make_to_vector(tokens):
    mean_of_vectors = []
    except_tokens = []
    for token in tokens:
        try:
            vector = w2v_model.wv.get_vector(token)
            mean_of_vectors.append(vector)
        except KeyError:
#             print("{} is not exist in the vocab".format(noun))
            except_tokens.append(token)
            continue
    try:
        mean_of_vectors = sum(mean_of_vectors) / len(mean_of_vectors)
        return mean_of_vectors
    except ZeroDivisionError:
        return np.nan

In [18]:
df['doc_vector'] = df['tokenized_text'].apply(make_to_vector)
df = df.dropna()
df = df.reset_index(drop=True)

In [19]:
vectors = np.zeros(128,)
for vector in df.doc_vector:
    vectors = np.vstack([vectors, vector])
vectors = vectors[1:, :]
vectors

array([[-0.00423659,  0.17780623,  0.20986421, ..., -0.09617404,
         0.2035376 ,  0.10911077],
       [-0.02205996,  0.17980617,  0.16847241, ..., -0.1201755 ,
         0.18646029,  0.11190627],
       [ 0.01685894,  0.18243246,  0.1859199 , ..., -0.1138395 ,
         0.18536329,  0.08026116],
       ...,
       [-0.05851585,  0.10892133,  0.23614064, ..., -0.08655355,
         0.1845157 ,  0.0366469 ],
       [ 0.03773687,  0.15562312,  0.19995259, ..., -0.09915192,
         0.21535033,  0.11777136],
       [ 0.05184415,  0.18287562,  0.21582051, ..., -0.09163125,
         0.23449181,  0.10614546]])

>ndarray 형식으로 예쁘게 만들어야 되는데 이런 식으로 밖에 못만드는게 내 실력 미달인듯... ㅠ

## Normalize extracted feature set
- 유사도 계산의 편의를 위해 0과 1사이의 값으로 정규화

In [20]:
from sklearn.preprocessing import Normalizer

normalizer = Normalizer()
norm_features=normalizer.fit_transform(vectors)
df_features = pd.DataFrame(norm_features,index=df.index.tolist())
df_features.tail()

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
19499,0.100223,-0.015523,0.125701,0.002736,0.059570,0.002465,0.083391,-0.009936,-0.154709,-0.087337,...,0.096420,-0.096232,0.033048,-0.001364,0.002103,-0.025324,0.006432,-0.128888,0.019098,0.054237
19500,0.039731,0.070507,0.139957,-0.108033,0.128847,0.151288,0.235687,0.082549,0.007348,-0.086715,...,0.137869,-0.073415,0.059888,-0.001680,-0.041405,0.054525,0.078836,-0.081503,0.169226,0.059253
19501,-0.036917,0.068717,0.148978,-0.012886,0.180859,0.128598,0.259999,0.023207,0.057743,-0.101745,...,0.139774,-0.063850,-0.009105,-0.066711,-0.019973,-0.037450,0.068126,-0.054605,0.116408,0.023120
19502,0.024387,0.100570,0.129218,-0.117767,0.168201,0.103840,0.169654,0.068522,-0.006473,-0.083712,...,0.139642,-0.099281,0.035922,-0.031027,-0.065497,0.019676,0.068444,-0.064076,0.139168,0.076109
19503,0.033040,0.116546,0.137542,-0.118049,0.148602,0.113015,0.199667,0.069674,0.019346,-0.087894,...,0.122310,-0.081414,0.061879,-0.019699,-0.061451,0.009648,0.083389,-0.058397,0.149441,0.067646


In [21]:
article = df_features.loc[0]
similarities=df_features.dot(article) # cosine similarity
top=similarities.nlargest(5)

texts = df.loc[top.index]['text'].tolist()

i = 0
print("<비교할 문서>")
for text in texts:
    df_idx = top.index[i]
    print('TITLE :'+str(df_idx)+" Similarities:"+ str(top[df_idx]))
    print(text+'\n')
    i = i+1

<비교할 문서>
TITLE :0 Similarities:1.0000000000000004
6살 남아 입니다 회사 복직 후 3살부터 주중에는 외할머니댁에서 2살 4살 터울의 사촌 형 누나와 함께 자랐어요 금요일 저녁에 데려와 주말에는 엄마 아빠와 생활하고 있구요 7세가 되는 내년이라도 아이를 데려오자는 아이 아빠의 의견과 유치원과 태권도 등 패턴이 적응되었고 형 누나들과 더 자랄 수 있게 초등학교 입학시기에 데리고 가는게 낫지 않겠냐는 외할머니 의견이 다릅니다 6세가 된 아이의 교육과 생활습관 형성이 이제는 할머니댁에서는 어려울 것 같다는 생각이 들고 하루하루 생각이 크고 자라는 아이를 주말에만 보는 저 또한 놀랄 때가 많거든요 하루라도 빨리 데려와야 할까요 초등학교 입학시기에 데려오는 편이 아이에게 좋을까요 매우 어려운 질문입니다 먼저 아이가 현재 잘 적응하고 있고 주변 양육환경 역시 아이가 성장하기에 별다른 문제가 없다 하니 고민이 더욱 크시리라 생각됩니다 우선 되어야하는 것은 아이의 의견입니다 6살이면 이러한 문제에 대하여 스스로 생각하고 의견을 전달할 수 있으므로 먼저 아이의 의견을 들어 주시기 바랍니다 아이는 부모님과 함께 사는 것이 좋습니다 그러나 부모님께서 두분 모두 직장에 나가시니 유치원 하원 후 아이를 돌보아 주실 분이 현재 함께 생활하고 계시는 할머님이시면 재고의 여지 없이 데려 오시는 것을 권유 드립니다 그러나 하원 후 할머님의 돌봄이 여의치 못할 경우 돌보아 줄 대상이 당분간 할머님댁에서 시간제로나마 아이를 돌보아 주어 아이와의 상호작용을 충분히 가진 후에 아이가 집으로 왔을 때 유치원 하원 후에 계속 돌보아 줄 수 있으면 좋습니다 이와 같이 먼저 아이가 집으로 온 후 부모님께서 퇴근하시기 전 까지 돌보아 줄 사람을 알아보시고 데려 오시는 것이 좋다는 생각이 듭니다

TITLE :17364 Similarities:0.9901152437787631
어린이교사학부모의 어려움 감싸는 따뜻한 울타리가 되고 싶어원 운영에서 교사가 가장 중요합니다 교

### 가상의 키워드를 입력하여 유사도 계산해보기
- 이번에는 챗봇처럼 짧은 문장(아마 키워드 위주)을 입력받아 유사도 계산을 해보자.
- word2vec vocabulary에 없는 단어를 입력받을 경우 에러가 발생하는 문제가 있다.

In [22]:
%%time
tokenizer = Mecab()

input_text = "중고 유모차 어떤 브랜드가 좋을까요? 추천 부탁해요 ㅇㅅㅇ!"

tokens = tokenizer.morphs(input_text)
input_vector = []

for token in tokens:
    try:
        word_vector = w2v_model.wv.get_vector(token)
        input_vector.append(word_vector)
    except:
        print("'{}'은(는) 제가 아직 잘 이해하지 못하겠어요ㅠㅠ ".format(token))
        continue

input_vector = sum(input_vector) / len(input_vector)
input_vector = input_vector.reshape(1, -1)
input_vector = normalizer.fit_transform(input_vector)
input_vector = input_vector.reshape(-1,)

similarities=df_features.dot(input_vector)
top=similarities.nlargest(5)

texts = df.loc[top.index]['text'].tolist()

i = 0
for text in texts:
    df_idx = top.index[i]
    print('\n'+'TITLE :'+str(df_idx)+" Similarities:"+ str(top[df_idx]))
    print(text+'\n')
    i = i+1

'?'은(는) 제가 아직 잘 이해하지 못하겠어요ㅠㅠ 
'ㅇㅅㅇ'은(는) 제가 아직 잘 이해하지 못하겠어요ㅠㅠ 
'!'은(는) 제가 아직 잘 이해하지 못하겠어요ㅠㅠ 

TITLE :2609 Similarities:0.8824055269028124
엄마들의 유모차 브랜드 순위 Best5 엄마들이 가장 많이 찾는유모차 순위 베스트 5 엄마들이 가장 많이 찾는 유모차 브랜드들 어떤 것들이 있을까요 디터가 후기 질문 그리고 엄마들의 추천 인터넷에서 많이 검색하는 키워드들을 바탕으로 유모차 브랜드 순위 베스트 5를 뽑아봤어요 아래 브랜드는 디럭스형 휴대형 절충형 유모차 구분 없이 전체 브랜드들 중 엄마들이 많이 찾아보고 궁금해 하는 브랜드 기준이니 각 유형별 유모차 순위를 보고 싶으시다면 아래 링크를 해 주세요 유모차의 모든 것 유모차의 종류 및 선택 가이드를 알고 싶다면 디럭스 유모차 선택 가이드 및 추천 유모차 절충형 유모차 선택 가이드 및 추천 유모차는 휴대형 유모차 선택 가이드 및 추천 유모차는 유모차 순위 BEST 5 그럼 엄마들이 가장 많이 찾는 유모차 브랜드 순위 베스트 5 보실까요 1위 #잉글레시나 유모차 이태리 명품 유모차 브랜드 잉글레시나가 역시나 단연 1위 잉글레시나 쿼드 트립 트릴로지 등 다양한 종류가 있죠 뛰어난 품질과 기술력은 물론이고 핸들링 때문에 더욱 사랑 받고 있어요 엄마들에겐 편리함을 아기들에겐 안전성과 편안함을 그럼 들은 후기는 어떨까요 들의 잉글레시나 유모차 구매후기 각각의 텍스트 시 상세후기를 볼 수 있어요 짱이맘찐 핸들링은 정말 최고 그런데 폴딩을 하면 의자와 분리를 해야 되어 그게 제일 불편해요미소천사태형맘 줄즈와 트릴로지 중앙에서 고민하다 선택 장점은 양대면 전환이 쉽고 개폐식 차양막이라는점 제일 좋은점은 폴딩이 엄청 편해요 단점은 83민준은서맘 휴대영으로 구입 민준이가 너무 좋아해요 진작 사줄걸 2위 #부가부유모차 네덜란드의 개성있는 유모차 부가부장소에 구애받지 않고 심지어 모래사장에서도 달릴 수 있는 유모차로 유명한 부가부는

```
tokenizer = Mecab()

while True:
    input_text = input("Input text:")
    if input_text == "exit":
        print("Bye")
        break
        
    tokens = tokenizer.morphs(input_text)
    input_vector = []
    
    for token in tokens:
        try:
            word_vector = w2v_model.wv.get_vector(token)
            input_vector.append(word_vector)
        except:
            print("'{}'은(는) 제가 아직 잘 이해하지 못하겠어요ㅠㅠ ".format(token))
            continue
    
    input_vector = sum(input_vector) / len(input_vector)
    input_vector = input_vector.reshape(1, -1)
    input_vector = normalizer.fit_transform(input_vector)
    input_vector = input_vector.reshape(-1,)
    
    similarities=df_features.dot(input_vector)
    top=similarities.nlargest(5)

    texts = df.loc[top.index]['text'].tolist()

    i = 0
    for text in texts:
        df_idx = top.index[i]
        print('\n'+'TITLE :'+str(df_idx)+" Similarities:"+ str(top[df_idx]))
        print(text+'\n')
        i = i+1
```

---

## Fasttext

In [23]:
fasttext_model = FastText.load('data/tokmom_fasttext.model')
df = pd.read_csv('data/text_for_word2vec.csv')
df = df.dropna()
df = df.reset_index(drop=True)

### Preprocessing for FastText 
- 한글을 자음, 모음 단위로 분해
- 단어 단위로 구분된 ndarray 자료구조로 정제

In [24]:
import re
from soynlp.hangle import decompose, compose

def remove_doublespace(s):
    doublespace_pattern = re.compile('\s+')
    return doublespace_pattern.sub(' ', s).strip()

def encode(s):
    def process(c):
        if c == ' ':
            return c
        jamo = decompose(c)
        # 'a' or 모음 or 자음
        if (jamo is None) or (jamo[0] == ' ') or (jamo[1] == ' '):
            return ' '
        base = jamo[0]+jamo[1]
        if jamo[2] == ' ':
            return base + '-'
        return base + jamo[2]

    s = ''.join(process(c) for c in s)
    return remove_doublespace(s).strip()

#     s = [process(c) for c in s]
#     return s

def decode(s):
    def process(t):
        assert len(t) % 3 == 0
        t_ = t.replace('-', ' ')
        chars = [tuple(t_[3*i:3*(i+1)]) for i in range(len(t_)//3)]
        recovered = [compose(*char) for char in chars]
        recovered = ''.join(recovered)
        return recovered

    return ' '.join(process(t) for t in sdf_features.split())

In [25]:
df['encoded'] = df['text'].apply(encode)

### Vectorizing

In [26]:
def jamo_to_vector(tokens):
    mean_of_vectors = []
    except_tokens = []
    tokens = tokens.split(' ') # 띄어쓰기 단위로 글자 구분을 해준다.
    for token in tokens:
        try:
            vector = fasttext_model.wv.get_vector(token)
            mean_of_vectors.append(vector)
        except KeyError:
            print("{} is not exist in the vocab".format(noun))
            except_tokens.append(token)
            continue
    try:
        mean_of_vectors = sum(mean_of_vectors) / len(mean_of_vectors)
        return mean_of_vectors
    except ZeroDivisionError:
        return np.nan

In [27]:
%%time
df['doc_vector'] = df['encoded'].apply(jamo_to_vector)
df = df.dropna()
df = df.reset_index(drop=True)

CPU times: user 54.9 s, sys: 0 ns, total: 54.9 s
Wall time: 54.9 s


In [28]:
vectors = np.zeros(100,)
for vector in df.doc_vector:
    vectors = np.vstack([vectors, vector])
vectors = vectors[1:, :]
vectors

array([[ 0.86658299, -0.56535929, -0.45894697, ...,  0.00341589,
        -0.33599198,  0.62156469],
       [ 0.53817666, -0.72349644, -0.85632437, ...,  0.05255881,
        -0.50502682,  0.70481634],
       [ 0.50858897, -0.60111189, -0.46155089, ..., -0.24129999,
        -0.67478687,  0.6221779 ],
       ...,
       [ 0.29718897, -0.43003976, -0.75820154, ..., -0.08352479,
        -0.68372095,  0.88945806],
       [ 0.41118783, -0.20348136, -1.08845186, ...,  0.0540762 ,
        -0.61487466,  0.9850949 ],
       [ 0.38487715, -0.41307026, -0.82243931, ..., -0.05015103,
        -0.7017765 ,  0.58764511]])

>예쁘게 만들어야 되는데 이런 식으로 밖에 못만드는게 내 실력 미달인듯... ㅠ

## Normalize extracted feature set
- 유사도 계산의 편의를 위해 0과 1사이의 값으로 정규화

In [29]:
normalizer = Normalizer()
norm_features = normalizer.fit_transform(vectors)
df_features = pd.DataFrame(norm_features,index=df.index.tolist())
df_features.tail()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
19500,-0.157133,0.189910,-0.188528,0.019966,-0.172055,-0.209940,-0.082700,0.022620,-0.019062,0.084801,...,-0.145142,0.096071,0.087845,0.030447,0.101301,0.090200,0.007959,-0.133222,0.048011,-0.011496
19501,0.068911,-0.042133,-0.182900,-0.050632,-0.176763,-0.126192,-0.103488,-0.125923,-0.049064,0.256410,...,-0.097004,0.031894,-0.137346,-0.075640,0.090801,0.080767,-0.024288,0.014092,-0.117959,0.122390
19502,0.040154,-0.058103,-0.102442,0.006907,-0.196867,-0.057263,-0.137394,-0.103822,-0.077189,0.193767,...,-0.177352,0.039965,-0.033948,-0.178392,0.086315,0.064732,0.037422,-0.011285,-0.092378,0.120176
19503,0.052673,-0.026066,-0.139431,0.013634,-0.144553,-0.092317,-0.157348,-0.014650,-0.072456,0.246368,...,-0.127140,0.018352,-0.122910,-0.136783,0.155050,0.061784,0.064725,0.006927,-0.078766,0.126191
19504,0.056155,-0.060268,-0.119996,0.012139,-0.234812,-0.071636,-0.119494,-0.088790,-0.074353,0.295101,...,-0.154728,0.073312,-0.139678,-0.085137,0.125316,0.041369,-0.025539,-0.007317,-0.102391,0.085739


In [30]:
article = df_features.loc[0]
print(article.shape, df_features.shape)
print(np.linalg.norm(article), np.linalg.norm(df_features, axis=1))
similarities=df_features.dot(article) # cosine similarity
top=similarities.nlargest(5)

texts = df.loc[top.index]['text'].tolist()

i = 0
print("<비교할 문서>")
for text in texts:
    df_idx = top.index[i]
    print('TITLE :'+str(df_idx)+" Similarities:"+ str(top[df_idx]))
    print(text+'\n')
    i = i+1

(100,) (19505, 100)
1.0 [1. 1. 1. ... 1. 1. 1.]
<비교할 문서>
TITLE :0 Similarities:1.0000000000000002
6살 남아 입니다 회사 복직 후 3살부터 주중에는 외할머니댁에서 2살 4살 터울의 사촌 형 누나와 함께 자랐어요 금요일 저녁에 데려와 주말에는 엄마 아빠와 생활하고 있구요 7세가 되는 내년이라도 아이를 데려오자는 아이 아빠의 의견과 유치원과 태권도 등 패턴이 적응되었고 형 누나들과 더 자랄 수 있게 초등학교 입학시기에 데리고 가는게 낫지 않겠냐는 외할머니 의견이 다릅니다 6세가 된 아이의 교육과 생활습관 형성이 이제는 할머니댁에서는 어려울 것 같다는 생각이 들고 하루하루 생각이 크고 자라는 아이를 주말에만 보는 저 또한 놀랄 때가 많거든요 하루라도 빨리 데려와야 할까요 초등학교 입학시기에 데려오는 편이 아이에게 좋을까요 매우 어려운 질문입니다 먼저 아이가 현재 잘 적응하고 있고 주변 양육환경 역시 아이가 성장하기에 별다른 문제가 없다 하니 고민이 더욱 크시리라 생각됩니다 우선 되어야하는 것은 아이의 의견입니다 6살이면 이러한 문제에 대하여 스스로 생각하고 의견을 전달할 수 있으므로 먼저 아이의 의견을 들어 주시기 바랍니다 아이는 부모님과 함께 사는 것이 좋습니다 그러나 부모님께서 두분 모두 직장에 나가시니 유치원 하원 후 아이를 돌보아 주실 분이 현재 함께 생활하고 계시는 할머님이시면 재고의 여지 없이 데려 오시는 것을 권유 드립니다 그러나 하원 후 할머님의 돌봄이 여의치 못할 경우 돌보아 줄 대상이 당분간 할머님댁에서 시간제로나마 아이를 돌보아 주어 아이와의 상호작용을 충분히 가진 후에 아이가 집으로 왔을 때 유치원 하원 후에 계속 돌보아 줄 수 있으면 좋습니다 이와 같이 먼저 아이가 집으로 온 후 부모님께서 퇴근하시기 전 까지 돌보아 줄 사람을 알아보시고 데려 오시는 것이 좋다는 생각이 듭니다

TITLE :18912 Similarities:0.9693972701129508
주변에 연

### 가상의 키워드를 입력하여 유사도 계산해보기
- 이번에는 챗봇처럼 짧은 문장(아마 키워드 위주)을 입력받아 유사도 계산을 해보자.
- Fasttext는 word2vec과 달리 단어를 입력받을 경우 에러가 발생하는 문제를 방지할 수 있다!

In [31]:
%%time
input_text = "중고 유모차 어떤 브랜드가 좋을까요? 추천 부탁해요 ㅇㅅㅇ!"

tokens = encode(input_text)
tokens = tokens.split(' ')

normalizer = Normalizer()
input_vector = np.array([fasttext_model.wv.get_vector(token) for token in tokens])vectorizer
input_vector = sum(input_vector) / len(input_vector)

input_vector = input_vector.reshape(1, -1)
input_vector = normalizer.fit_transform(input_vector)
input_vector = input_vector.reshape(-1,)

similarities=df_features.dot(input_vector)
top=similarities.nlargest(5)

texts = df.loc[top.index]['text'].tolist()

i = 0
for text in texts:
    df_idx = top.index[i]
    print('\n'+'TITLE :'+str(df_idx)+" Similarities:"+ str(top[df_idx]))
    print(text+'\n')
    i = i+1


TITLE :2510 Similarities:0.7886206098571207
신생아 선물 추천 뭘로 하면 좋아할까 뭘 사주면 좋아하지 신생아 선물 추천 친구가 혹은 동료가 아기를 낳았는데 무슨 선물을 해줘야할지 고민이라면 이미 출산준비물은 다 샀을테고 고가의 제품은 부담이 되고 비싸지 않으면서 겹치지 않을만한 선물들을 골라봤어요 신생아 선물 하면 딸랑이 아기 내의 기저기만 생각나시죠 경쟁력 없는 선물은 이제 그만겹치지 않는 나만의 아이템으로 선물해 보세요 애착인형 애착 인형 중에서도 엄마들사이에서 단연 최고의 인기를 끌고 있는 건 바로 젤리켓 애착인형 아이들의 애착인형이 더 많이 보고 싶다면 여기를 하세요 물티슈엄마들에게는 큰 고민거리인 물티슈또한 매월 한박스씩 사용하게 되는 물티슈 이왕이면 안정성과 보습효과까지 갖춘 간나나기 물티슈 어때요 물티슈 선물을 고민 중이라면 흔한 물티슈 대신 좋은 물티슈로 목욕장난감집집마다 하나씩은 가지고 있는 목욕장난감그 중 옥소도트 고래 목욕정리함은 내 돈 주고 사긴 아깝지만 선물로 들어오면 좋아할만한 제품 기저귀가방요즘 엄마들의 패션아이템 중 하나인 기저귀 가방그 중 포브 미니프리저는 보온보냉 그리고 방수기능에다가 이유식 분유까지 수납이 가능한 다재다능 가방요 아이템도 선물받기 좋은 아이템기저귀 가방 구매가이드와 제품 추천이 궁금하다면 여기를 하세요 치발기아기 이나기전부터 사용하는 치발기모든 아기들에게 필요한 제품이자 약 4 5개월부터 사용하기 때문에 신생아때는 잘 구비하지 않는 제품그 중 누비 스퀴즈는 귀여운 디자인과 안정성있는 소재를 사용해 선물받는 엄마들이 좋아할만한 제품 놀이방 매트정말 친한친구라면 선물할만한 놀이방 매트그 중 폴더웨이는 남자아이들이 좋아할만한 디자인을 갖추고 있어 남자 신생아 아기 선물로 딱 아기화장품화장품 성분때문에 예민한 요즘 알티야 오가닉스 스킨케어는 USDA유기농 인증은 물론이고 3년 연속 영구 베이비프리마 어워드 수상으로 선물세트 준비할때 딱 좋은 제품


TITLE :2609 Simila

```
while True:
    input_text = input("Input text:")
    if input_text == "exit":
        print("Bye")
        break
        
    tokens = encode(input_text)
    tokens = tokens.split(' ')

    normalizer = Normalizer()
    input_vector = np.array([fasttext_model.wv.get_vector(token) for token in tokens])
    input_vector = sum(input_vector) / len(input_vector)
  
    input_vector = input_vector.reshape(1, -1)
    input_vector = normalizer.fit_transform(input_vector)
    input_vector = input_vector.reshape(-1,)

    similarities=df_features.dot(input_vector)
    top=similarities.nlargest(5)

    texts = df.loc[top.index]['text'].tolist()

    i = 0
    for text in texts:
        df_idx = top.index[i]
        print('\n'+'TITLE :'+str(df_idx)+" Similarities:"+ str(top[df_idx]))
        print(text+'\n')
        i = i+1
```

---

### Fasttext
- 자모 단위가 아닌 글자 단위로 했을 때

In [32]:
fasttext_model = FastText.load('data/tokmom_token_fasttext.model')
df = pd.read_csv('data/text_for_word2vec.csv')
df = df.dropna()
df = df.reset_index(drop=True)

### Preprocessing for FastText 
- 한글을 글자 단위로 분해
- 단어 단위로 구분된 ndarray 자료구조로 정제

In [33]:
df['encoded'] = df['text'].apply(tokenize)

### Vectorizing

In [34]:
def make_to_vector(tokens):
    mean_of_vectors = []
    except_tokens = []
    for token in tokens:
        try:
            vector = fasttext_model.wv.get_vector(token)
            mean_of_vectors.append(vector)
        except KeyError:
#             print("{} is not exist in the vocab".format(noun))
            except_tokens.append(token)
            continue
    try:
        mean_of_vectors = sum(mean_of_vectors) / len(mean_of_vectors)
        return mean_of_vectors
    except ZeroDivisionError:
        return np.nan

In [35]:
%%time
df['doc_vector'] = df['encoded'].apply(make_to_vector)
df = df.dropna()
df = df.reset_index(drop=True)

CPU times: user 18.5 s, sys: 455 µs, total: 18.5 s
Wall time: 18.5 s


In [36]:
vectors = np.zeros(100,)
for vector in df.doc_vector:
    vectors = np.vstack([vectors, vector])
vectors = vectors[1:, :]
vectors

array([[ 0.24673061, -0.27795058,  0.24404778, ...,  1.11242867,
         1.08872926,  0.48421091],
       [ 0.01572265, -0.09072687,  0.1175985 , ...,  0.94063079,
         1.27431548,  0.4101868 ],
       [ 0.12905237,  0.04540736,  0.11949921, ...,  1.09621668,
         1.10972857,  0.29464814],
       ...,
       [-0.20894928, -0.15436162,  0.18836324, ...,  1.18231559,
         1.03075528,  0.16409379],
       [ 0.04918776, -0.01805375,  0.07689283, ...,  1.29379928,
         1.25442481,  0.10543385],
       [-0.05928177, -0.18643324,  0.17147981, ...,  1.32759964,
         1.09847867,  0.12419592]])

>예쁘게 만들어야 되는데 이런 식으로 밖에 못만드는게 내 실력 미달인듯... ㅠ

## Normalize extracted feature set
- 유사도 계산의 편의를 위해 0과 1사이의 값으로 정규화

In [37]:
normalizer = Normalizer()
norm_features = normalizer.fit_transform(vectors)
df_features = pd.DataFrame(norm_features,index=df.index.tolist())
df_features.tail()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
19500,-0.082703,-0.107066,0.020534,-0.013594,0.011498,0.011114,-0.008895,0.004951,-0.031599,0.098213,...,0.229741,-0.229231,-0.021993,0.069794,0.057802,0.117444,-0.019022,0.094695,-0.264936,0.100297
19501,-0.013324,-0.043249,0.056210,0.023133,-0.051027,0.018710,-0.152465,-0.099115,-0.187265,0.053776,...,0.152398,-0.009613,0.017098,-0.036609,0.090501,0.225238,-0.139409,0.236008,0.175327,0.044390
19502,-0.035021,-0.025872,0.031570,0.034131,-0.015111,-0.030294,-0.204873,-0.154331,-0.142759,-0.012280,...,0.220279,-0.101568,0.006249,-0.106006,0.069603,0.127194,-0.097784,0.198160,0.172758,0.027503
19503,0.007437,-0.002730,0.011626,0.006192,-0.047806,-0.003482,-0.192131,-0.096706,-0.142590,0.030369,...,0.262496,-0.007289,0.031681,-0.009989,0.023988,0.201436,-0.076635,0.195617,0.189663,0.015941
19504,-0.009856,-0.030996,0.028509,0.039489,-0.057810,0.005713,-0.177771,-0.106272,-0.183521,0.011650,...,0.219025,0.001136,-0.027511,-0.026783,0.049193,0.167402,-0.071403,0.220721,0.182628,0.020648


In [38]:
article = df_features.loc[0]
print(article.shape, df_features.shape)
print(np.linalg.norm(article), np.linalg.norm(df_features, axis=1))
similarities=df_features.dot(article) # cosine similarity
top=similarities.nlargest(5)

texts = df.loc[top.index]['text'].tolist()

i = 0
print("<비교할 문서>")
for text in texts:
    df_idx = top.index[i]
    print('TITLE :'+str(df_idx)+" Similarities:"+ str(top[df_idx]))
    print(text+'\n')
    i = i+1

(100,) (19505, 100)
1.0 [1. 1. 1. ... 1. 1. 1.]
<비교할 문서>
TITLE :0 Similarities:1.0000000000000002
6살 남아 입니다 회사 복직 후 3살부터 주중에는 외할머니댁에서 2살 4살 터울의 사촌 형 누나와 함께 자랐어요 금요일 저녁에 데려와 주말에는 엄마 아빠와 생활하고 있구요 7세가 되는 내년이라도 아이를 데려오자는 아이 아빠의 의견과 유치원과 태권도 등 패턴이 적응되었고 형 누나들과 더 자랄 수 있게 초등학교 입학시기에 데리고 가는게 낫지 않겠냐는 외할머니 의견이 다릅니다 6세가 된 아이의 교육과 생활습관 형성이 이제는 할머니댁에서는 어려울 것 같다는 생각이 들고 하루하루 생각이 크고 자라는 아이를 주말에만 보는 저 또한 놀랄 때가 많거든요 하루라도 빨리 데려와야 할까요 초등학교 입학시기에 데려오는 편이 아이에게 좋을까요 매우 어려운 질문입니다 먼저 아이가 현재 잘 적응하고 있고 주변 양육환경 역시 아이가 성장하기에 별다른 문제가 없다 하니 고민이 더욱 크시리라 생각됩니다 우선 되어야하는 것은 아이의 의견입니다 6살이면 이러한 문제에 대하여 스스로 생각하고 의견을 전달할 수 있으므로 먼저 아이의 의견을 들어 주시기 바랍니다 아이는 부모님과 함께 사는 것이 좋습니다 그러나 부모님께서 두분 모두 직장에 나가시니 유치원 하원 후 아이를 돌보아 주실 분이 현재 함께 생활하고 계시는 할머님이시면 재고의 여지 없이 데려 오시는 것을 권유 드립니다 그러나 하원 후 할머님의 돌봄이 여의치 못할 경우 돌보아 줄 대상이 당분간 할머님댁에서 시간제로나마 아이를 돌보아 주어 아이와의 상호작용을 충분히 가진 후에 아이가 집으로 왔을 때 유치원 하원 후에 계속 돌보아 줄 수 있으면 좋습니다 이와 같이 먼저 아이가 집으로 온 후 부모님께서 퇴근하시기 전 까지 돌보아 줄 사람을 알아보시고 데려 오시는 것이 좋다는 생각이 듭니다

TITLE :2143 Similarities:0.972685979454132
워킹맘의 고민

### 가상의 키워드를 입력하여 유사도 계산해보기
- 이번에는 챗봇처럼 짧은 문장(아마 키워드 위주)을 입력받아 유사도 계산을 해보자.
- Fasttext는 word2vec과 달리 단어를 입력받을 경우 에러가 발생하는 문제를 방지할 수 있다!

In [39]:
%%time
input_text = "중고 유모차 어떤 브랜드가 좋을까요? 추천 부탁해요 ㅇㅅㅇ!"

tokens = tokenize(input_text)

input_vector = np.array([fasttext_model.wv.get_vector(token) for token in tokens])
input_vector = sum(input_vector) / len(input_vector)

input_vector = input_vector.reshape(1, -1)
input_vector = normalizer.fit_transform(input_vector)
input_vector = input_vector.reshape(-1,)

similarities=df_features.dot(input_vector)
top=similarities.nlargest(5)

texts = df.loc[top.index]['text'].tolist()

i = 0
for text in texts:
    df_idx = top.index[i]
    print('\n'+'TITLE :'+str(df_idx)+" Similarities:"+ str(top[df_idx]))
    print(text+'\n')
    i = i+1


TITLE :2600 Similarities:0.7428661238843963
주방놀이 장난감 고르는 법과 추천제품 주방놀이세트로 살까 원목주방놀이로 살까 주방놀이 소품만 사야하나요 스텝투 주방놀이 키즈크래프트 주방놀이 테팔 주방몰이 다양한데 뭘 사야하지 주방놀이 장난감 고르는 법과 추천제품엄마가 더 설레는 주방놀이 장난감 고르기 그런데 종류가 많아도 정말 너무 많죠 제품에 따라 대 로 구매 가능합니다 이케아 둑티그 주방놀이 구매하러가기리틀타익스 싱크앤스토브 구매하러가기미키 주방놀이 구매하러가기 어때요 주방놀이 하나로도 정말 다양한 브랜드 다양한 제품이 있죠 브랜드 마다 제품군도 가격대도 다양하니 하나하나 다 살펴보긴 힘들거에요 이런 걸 찾는 다면 요 브랜드를 찾아보세요 하고 알려드렸으니 이젠 요기서 맘에 드는 제품위주로 콕콕 찍어서 알아보면 되겠죠 여러분은 어떤 주방놀이 제품으로 고민하고 계세요 어떤 주방놀이를 사주셨나요 함께 얘기 나눠주세요 여러분의 생생한 사용기와 추천을 기다립니다 소피마르소 님의 숲소리 주방놀이 사용후기도 참고해보세요 talk view 681237


TITLE :2609 Similarities:0.7423392952705492
엄마들의 유모차 브랜드 순위 Best5 엄마들이 가장 많이 찾는유모차 순위 베스트 5 엄마들이 가장 많이 찾는 유모차 브랜드들 어떤 것들이 있을까요 디터가 후기 질문 그리고 엄마들의 추천 인터넷에서 많이 검색하는 키워드들을 바탕으로 유모차 브랜드 순위 베스트 5를 뽑아봤어요 아래 브랜드는 디럭스형 휴대형 절충형 유모차 구분 없이 전체 브랜드들 중 엄마들이 많이 찾아보고 궁금해 하는 브랜드 기준이니 각 유형별 유모차 순위를 보고 싶으시다면 아래 링크를 해 주세요 유모차의 모든 것 유모차의 종류 및 선택 가이드를 알고 싶다면 디럭스 유모차 선택 가이드 및 추천 유모차 절충형 유모차 선택 가이드 및 추천 유모차는 휴대형 유모차 선택 가이드 및 추천 유모차는 유모차 순위 BEST 5 그럼 엄마들이 가장 많이 찾는 유모

```
while True:
    input_text = input("Input text:")
    if input_text == "exit":
        print("Bye")
        break
        
    tokens = tokenize(input_text)

    input_vector = np.array([fasttext_model.wv.get_vector(token) for token in tokens])
    input_vector = sum(input_vector) / len(input_vector)
  
    input_vector = input_vector.reshape(1, -1)
    input_vector = normalizer.fit_transform(input_vector)
    input_vector = input_vector.reshape(-1,)

    similarities=df_features.dot(input_vector)
    top=similarities.nlargest(5)

    texts = df.loc[top.index]['text'].tolist()

    i = 0
    for text in texts:
        df_idx = top.index[i]
        print('\n'+'TITLE :'+str(df_idx)+" Similarities:"+ str(top[df_idx]))
        print(text+'\n')
        i = i+1
```